# Predicting System Failure

### Description
Model generation for supervised models (GBM, deepnet, logistic regression) to be used in tests.

In this Jupyter notebook, we will examine sensory dataset from Nasa where we will create a model in order to predict system failure. This notebook can be generalized to any sensory driven data, such as manufacturing.

In [1]:
import numpy as np
import pandas as pd
# import seaborn as sns

In [2]:
import sys
print(sys.version)

3.8.10 (default, Mar 15 2022, 12:22:08) 
[GCC 9.4.0]


In [3]:
import os
ARTIFACTS_PATH = '../../artifacts/bigml/'
os.makedirs(ARTIFACTS_PATH, exist_ok=True) # Create path if not exists

### Save our BigML Username and Api Key to our environment to access the API

In [4]:
import os
os.environ['BIGML_USERNAME'] = "elizaglez"
os.environ['BIGML_API_KEY'] = "2064c7cbf0a35661eb33602d6ecd264aecdfd520"
os.environ['BIGML_CRED'] = "username=$BIGML_USERNAME;api_key=$BIGML_API_KEY"
os.environ['BIGML_PROJECT'] = "project/59ef803eaf447f0b1d001156"

### Creat our main API object that all the main functions will utilize.

In [5]:
from bigml.api import BigML
api = BigML(project="project/59ef803eaf447f0b1d001156")

In [6]:
data_source = api.create_source("../../data/failure/failure_small.csv")
api.ok(data_source)

True

In [7]:
data = api.create_dataset(data_source)
api.ok(data)

True

## Test-Train Split

In [8]:
train_dataset = api.create_dataset(
    data, {"name": "Engine Failure | Training",
                     "sample_rate": 0.8, "seed": "my seed"})
api.ok(train_dataset)

True

In [9]:
test_dataset = api.create_dataset(
    data, {"name": "Engine Failure | Test",
                     "sample_rate": 0.8, "seed": "my seed",
                     "out_of_bag": True})
api.ok(test_dataset)

True

## Creating Models

### Deepnet

In [10]:
model_name = "deepnet_bigml"
desc = "bigml deepnet test for dataset failure"
model_out_file_name = model_name + ".json"
model_out_path = os.path.join(ARTIFACTS_PATH, model_out_file_name)

In [11]:
deepnet_model = api.create_deepnet(train_dataset, {
    "name" : model_name,
    "tags" : [model_name],
    "excluded_fields": ["RUL","unit_number"],
    "objective_field": "failure_next_30_cycles"})

api.ok(deepnet_model)

True

In [12]:
pred = api.create_evaluation(deepnet_model,test_dataset)
api.ok(pred)

True

In [13]:
api.export_last(model_name,
                resource_type="deepnet",
                filename=model_out_path)


'../../artifacts/bigml//deepnet_bigml.json'

In [14]:
input_data = {"unit_number":1,"time_cycles": 1,"op_setting_1": -0.0007,"op_setting_2": -0.0004,"op_setting_3": 100,"sensory_measure_1": 518.67,"sensory_measure_2": 641.82,"sensory_measure_3": 1589.7, "sensory_measure_4": 1400.6,"sensory_measure_5": 14.62, "sensory_measure_6": 21.61,"sensory_measure_7": 554.36,"sensory_measure_8": 2388.06,"sensory_measure_9": 9046.19, "sensory_measure_10": 1.3,"sensory_measure_11": 47.47,"sensory_measure_12": 521.66,"sensory_measure_13": 2388.02,"sensory_measure_14": 8.4195,"sensory_measure_15": 0.03, "sensory_measure_16": 39.06,"sensory_measure_17": 100.0,"sensory_measure_18": 2388,"sensory_measure_19": 392,"sensory_measure_20": 0.03,"sensory_measure_21": 23.419, "prev_5_mean": 1439.2204047619048,"prev_5_std": 2464.9756804296103}

out =  api.create_prediction(deepnet_model,input_data)
print(out["object"]["output"])

False


In [15]:
input_data = {"unit_number":260,"time_cycles":315,"op_setting_1":25.0033,"op_setting_2":0.622,"op_setting_3":60,"sensory_measure_1":462.54,"sensory_measure_2":537.84,"sensory_measure_3":1272.95,"sensory_measure_4":1066.3,"sensory_measure_5":7.05,"sensory_measure_6":9.04,"sensory_measure_7":174.81,"sensory_measure_8":1917.36,"sensory_measure_9":8112.63,"sensory_measure_10":0.94,"sensory_measure_11":37.32,"sensory_measure_12":164.37,"sensory_measure_13":2030.35,"sensory_measure_14":7972.47,"sensory_measure_15":11.0537,"sensory_measure_16":0.02,"sensory_measure_17":311,"sensory_measure_18":1915,"sensory_measure_19":84.93,"sensory_measure_20":14.05,"sensory_measure_21":8.3729,"prev_5_mean":1255.9403841269843,"prev_5_std":2306.2312965309643}

out =  api.create_prediction(deepnet_model,input_data)
print(out["object"]["output"])

True


### Logistic Regresssion

In [16]:
model_name = "logreg_bigml"
model_out_file_name = model_name + ".json"
model_out_path = os.path.join(ARTIFACTS_PATH, model_out_file_name)

In [17]:
logreg_model = api.create_logistic_regression(train_dataset, {
    "name" : model_name,
    "tags" : [model_name],
    "excluded_fields": ["RUL","unit_number"],
    "objective_field": "failure_next_30_cycles"})

api.ok(logreg_model)

True

In [18]:
api.export_last(model_name,
                resource_type="logisticregression",
                filename=model_out_path)

'../../artifacts/bigml//logreg_bigml.json'

In [19]:
input_data = {"unit_number":1,"time_cycles": 1,"op_setting_1": -0.0007,"op_setting_2": -0.0004,"op_setting_3": 100,"sensory_measure_1": 518.67,"sensory_measure_2": 641.82,"sensory_measure_3": 1589.7, "sensory_measure_4": 1400.6,"sensory_measure_5": 14.62, "sensory_measure_6": 21.61,"sensory_measure_7": 554.36,"sensory_measure_8": 2388.06,"sensory_measure_9": 9046.19, "sensory_measure_10": 1.3,"sensory_measure_11": 47.47,"sensory_measure_12": 521.66,"sensory_measure_13": 2388.02,"sensory_measure_14": 8.4195,"sensory_measure_15": 0.03, "sensory_measure_16": 39.06,"sensory_measure_17": 100.0,"sensory_measure_18": 2388,"sensory_measure_19": 392,"sensory_measure_20": 0.03,"sensory_measure_21": 23.419, "prev_5_mean": 1439.2204047619048,"prev_5_std": 2464.9756804296103}

out =  api.create_prediction(logreg_model,input_data)
print(out["object"]["output"])

True


In [20]:
input_data = {"unit_number":260,"time_cycles":315,"op_setting_1":25.0033,"op_setting_2":0.622,"op_setting_3":60,"sensory_measure_1":462.54,"sensory_measure_2":537.84,"sensory_measure_3":1272.95,"sensory_measure_4":1066.3,"sensory_measure_5":7.05,"sensory_measure_6":9.04,"sensory_measure_7":174.81,"sensory_measure_8":1917.36,"sensory_measure_9":8112.63,"sensory_measure_10":0.94,"sensory_measure_11":37.32,"sensory_measure_12":164.37,"sensory_measure_13":2030.35,"sensory_measure_14":7972.47,"sensory_measure_15":11.0537,"sensory_measure_16":0.02,"sensory_measure_17":311,"sensory_measure_18":1915,"sensory_measure_19":84.93,"sensory_measure_20":14.05,"sensory_measure_21":8.3729,"prev_5_mean":1255.9403841269843,"prev_5_std":2306.2312965309643}

out =  api.create_prediction(logreg_model,input_data)
print(out["object"]["output"])

True


### Model (GBM)

In [21]:
model_name = "gbm_bigml"
desc = "bigml gbm test for dataset failure"
model_out_file_name = model_name + ".json"
model_out_path = os.path.join(ARTIFACTS_PATH, model_out_file_name)

In [22]:
gbm_model = api.create_model(train_dataset, {
    "name" : model_name,
    "tags" : [model_name],
    "excluded_fields": ["RUL","unit_number"],
    "objective_field": "failure_next_30_cycles"})

api.ok(gbm_model)

True

In [23]:
api.export_last("gbm_model",
                resource_type="model",
                filename=model_out_path)


'../../artifacts/bigml//gbm_bigml.json'

In [24]:
input_data = {"unit_number":1,"time_cycles": 1,"op_setting_1": -0.0007,"op_setting_2": -0.0004,"op_setting_3": 100,"sensory_measure_1": 518.67,"sensory_measure_2": 641.82,"sensory_measure_3": 1589.7, "sensory_measure_4": 1400.6,"sensory_measure_5": 14.62, "sensory_measure_6": 21.61,"sensory_measure_7": 554.36,"sensory_measure_8": 2388.06,"sensory_measure_9": 9046.19, "sensory_measure_10": 1.3,"sensory_measure_11": 47.47,"sensory_measure_12": 521.66,"sensory_measure_13": 2388.02,"sensory_measure_14": 8.4195,"sensory_measure_15": 0.03, "sensory_measure_16": 39.06,"sensory_measure_17": 100.0,"sensory_measure_18": 2388,"sensory_measure_19": 392,"sensory_measure_20": 0.03,"sensory_measure_21": 23.419, "prev_5_mean": 1439.2204047619048,"prev_5_std": 2464.9756804296103}

out =  api.create_prediction(gbm_model,input_data)
print(out["object"]["output"])

False


In [25]:
input_data = {"unit_number":260,"time_cycles":315,"op_setting_1":25.0033,"op_setting_2":0.622,"op_setting_3":60,"sensory_measure_1":462.54,"sensory_measure_2":537.84,"sensory_measure_3":1272.95,"sensory_measure_4":1066.3,"sensory_measure_5":7.05,"sensory_measure_6":9.04,"sensory_measure_7":174.81,"sensory_measure_8":1917.36,"sensory_measure_9":8112.63,"sensory_measure_10":0.94,"sensory_measure_11":37.32,"sensory_measure_12":164.37,"sensory_measure_13":2030.35,"sensory_measure_14":7972.47,"sensory_measure_15":11.0537,"sensory_measure_16":0.02,"sensory_measure_17":311,"sensory_measure_18":1915,"sensory_measure_19":84.93,"sensory_measure_20":14.05,"sensory_measure_21":8.3729,"prev_5_mean":1255.9403841269843,"prev_5_std":2306.2312965309643}

out =  api.create_prediction(gbm_model,input_data)
print(out["object"]["output"])

True
